## Optimization

In [1]:
from autointent.nodes.optimization import NodeOptimizer

In [2]:
import logging

logger = logging.getLogger(__name__)

In [3]:
from autointent import Context
from autointent.pipeline.optimization.utils import get_run_name, load_data, get_db_dir


run_name = get_run_name("multiclass-cpu")
db_dir = get_db_dir("", run_name)

data = load_data("/home/voorhs/repos/AutoIntent/tests/minimal_optimization/data/clinc_subset.json", multilabel=False)
context = Context(
    multiclass_intent_records=data,
    multilabel_utterance_records=[],
    test_utterance_records=[],
    device="cpu",
    mode="multiclass",
    multilabel_generation_config="",
    db_dir=db_dir,
    regex_sampling=0,
    seed=0,
    dump_dir="modules_dumps_multiclass"
)

### Retrieval

In [4]:
retrieval_optimizer_config = {
    'metric': 'retrieval_hit_rate_intersecting',
    'node_type': 'retrieval',
    'search_space': [
        {
            'k': [10],
            'model_name': ['deepvk/USER-bge-m3'],
            'module_type': 'vector_db'
        }
    ]
}

In [5]:
retrieval_optimizer = NodeOptimizer.from_dict_config(retrieval_optimizer_config)

In [6]:
retrieval_optimizer.fit(context)

### Scoring

In [7]:
scoring_optimizer_config = {
    'metric': 'scoring_roc_auc',
    'node_type': 'scoring',
    'search_space': [
        {
            'k': [3],
            'module_type': 'knn',
            'weights': ['uniform', 'distance', 'closest']
        },
        {
            'module_type': 'linear'
        },
        # {
        #     "module_type": "mlknn",
        #     "k": [5]
        # },
        {
            "module_type": "dnnc",
            "model_name": ["cross-encoder/ms-marco-MiniLM-L-6-v2"],
            "k": [3],
            "train_head": [False, True]
        }
    ]
}

In [8]:
scoring_optimizer = NodeOptimizer.from_dict_config(scoring_optimizer_config)

In [9]:
scoring_optimizer.fit(context)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### Prediction

In [10]:
prediction_optimizer_config = {
    'metric': 'prediction_accuracy',
    'node_type': 'prediction',
    'search_space': [
        {
            'module_type': 'threshold',
            'thresh': [0.5]
        },
        {
            'module_type': 'tunable',
            'n_trials': [None, 3]
        },
        {
            'module_type': 'argmax',
        },
        {
            'module_type': 'jinoos',
        },
    ]
}

In [11]:
prediction_optimizer = NodeOptimizer.from_dict_config(prediction_optimizer_config)

In [12]:
prediction_optimizer.fit(context)

[I 2024-10-15 21:31:08,782] A new study created in memory with name: no-name-75e8efe0-2c53-434e-a110-e2d77bdac59a
Your data contains out-of-scope utterances, but ArgmaxPredictor cannot detect them. Consider different predictor


### check result

In [13]:
context.optimization_info.dump_evaluation_results()

{'metrics': {'regexp': [],
  'retrieval': [0.5],
  'scoring': [1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0],
  'prediction': [0.5714285714285714,
   0.7142857142857143,
   0.7142857142857143,
   0.5714285714285714,
   0.7142857142857143]},
 'configs': {'regexp': [],
  'retrieval': [{'module_type': 'vector_db',
    'module_params': {'k': 10, 'model_name': 'deepvk/USER-bge-m3'},
    'metric_name': 'retrieval_hit_rate_intersecting',
    'metric_value': 0.5,
    'module_dump_dir': 'modules_dumps_multiclass/retrieval/vector_db/comb_0'}],
  'scoring': [{'module_type': 'knn',
    'module_params': {'k': 3, 'weights': 'uniform'},
    'metric_name': 'scoring_roc_auc',
    'metric_value': 1.0,
    'module_dump_dir': 'modules_dumps_multiclass/scoring/knn/comb_0'},
   {'module_type': 'knn',
    'module_params': {'k': 3, 'weights': 'distance'},
    'metric_name': 'scoring_roc_auc',
    'metric_value': 1.0,
    'module_dump_dir': 'modules_dumps_multiclass/scoring/knn/comb_1'},
   {'module_type': 'knn

## Inference

In [14]:
from autointent.nodes import InferenceNode

### Retrieval

In [15]:
import gc
import torch


for trial in context.optimization_info.trials.retrieval:
    print(f"\n==== {trial.module_type} ====\n")
    config = dict(
        node_type="retrieval",
        module_type=trial.module_type,
        module_config=trial.module_params,
        load_path=trial.module_dump_dir,
    )
    node = InferenceNode(**config)
    labels, distances, texts = node.module.predict(["hello", "world"])
    print(labels[0], distances[0], texts[0])
    node.module.clear_cache()
    gc.collect()
    torch.cuda.empty_cache()


==== vector_db ====

[2, 2, 2, 2, 1, 1, 1, 0, 1, 0] [np.float32(0.477605), np.float32(0.4929347), np.float32(0.49597514), np.float32(0.50022346), np.float32(0.553351), np.float32(0.55573064), np.float32(0.5758226), np.float32(0.60500944), np.float32(0.6174678), np.float32(0.67098904)] ['please set an alarm for mid day', 'set my alarm for getting up', 'make sure my alarm is set for three thirty in the morning', 'have an alarm set for three in the morning', 'i am nost sure why my account is blocked', 'i think my account is blocked but i do not know the reason', 'can you tell me why is my bank account frozen', 'is it possible to make a reservation at redrobin', 'why is there a hold on my american saving bank account', 'does redrobin take reservations']


### Scoring

In [16]:
for trial in context.optimization_info.trials.scoring:
    print(f"\n==== {trial.module_type} ====\n")
    config = dict(
        node_type="scoring",
        module_type=trial.module_type,
        module_config=trial.module_params,
        load_path=trial.module_dump_dir,
    )
    node = InferenceNode(**config)
    scores = node.module.predict(["hello", "world"])
    print(scores)
    node.module.clear_cache()
    gc.collect()
    torch.cuda.empty_cache()


==== knn ====

[[0. 0. 1.]
 [0. 0. 1.]]

==== knn ====

[[0. 0. 1.]
 [0. 0. 1.]]

==== knn ====

[[0.         0.         0.7611975 ]
 [0.         0.         0.74537146]]

==== linear ====

[[0.21546907 0.33743832 0.44709261]
 [0.22099187 0.3357609  0.44324723]]

==== dnnc ====

[[  0.           0.         -10.27686977]
 [  0.           0.         -10.98405933]]

==== dnnc ====

[[0.         0.         0.18697797]
 [0.         0.         0.07185777]]


### Prediction

In [17]:
import numpy as np


for trial in context.optimization_info.trials.prediction:
    print(f"\n==== {trial.module_type} ====\n")
    config = dict(
        node_type="prediction",
        module_type=trial.module_type,
        module_config=trial.module_params,
        load_path=trial.module_dump_dir,
    )
    node = InferenceNode(**config)
    scores = node.module.predict(np.array([[0.27486506, 0.31681463, 0.37459106], [0.2769358,  0.31536099, 0.37366978]]))
    print(scores)
    node.module.clear_cache()
    gc.collect()
    torch.cuda.empty_cache()


==== threshold ====

[-1 -1]

==== tunable ====

[-1 -1]

==== tunable ====

[-1 -1]

==== argmax ====

[2 2]

==== jinoos ====

[-1 -1]
